# Task 2b: Abstractive Text Summarization

In this notebook, you manage challenges arising in large document summarization - input text can exceed model context lengths, generate hallucinated outputs, or trigger out-of-memory errors.

To mitigate these issues, this notebook demonstrates an architecture using prompt chunking and chaining with the [LangChain](https://python.langchain.com/docs/get_started/introduction.html) framework, a toolkit enabling applications leveraging language models.

You explore an approach addressing scenarios when user documents surpass token limits. Chunking splits documents into segments under context length thresholds before sequentially feeding them to models. This chains prompts across chunks, retaining prior context. You apply this approach to summarize call transcripts, meetings transcripts, books, articles, blog posts, and other relevant content.

## Task 2b.1: Environment setup

In this task, you set up your environment.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Task 2b.2: Summarize long text 

### Configuring LangChain with Boto3

In this task, you need to specify the LLM for the LangChain Bedrock class, and can pass arguments for inference.

In [ ]:
# model configuration
from langchain_aws import BedrockLLM
modelId = "amazon.titan-text-premier-v1:0"
llm = BedrockLLM(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 2048,
        "temperature": 0,
        "topP": 1
    },
    client=bedrock_client
)

## Task 2b.3: Loading a text file with many tokens

In this task, you can find a text file of [Amazon's CEO letter to shareholders in 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) in the letters directory. The following cell loads the text file and counts the number of tokens. You will see a warning indicating the number of tokens in the text file exceeds the maximum number of tokens for this model.

In [ ]:
#get tokens
shareholder_letter = "../letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** You can safely ignore the warnings and proceed to next cell.

## Task 2b.4: Splitting the long text into chunks

In this task, you split the text into smaller chunks because it is too long to fit in the prompt. `RecursiveCharacterTextSplitter` in LangChain supports splitting long text into chunks recursively until the size of each chunk becomes smaller than `chunk_size`. A text is separated with `separators=["\n\n", "\n"]` into chunks, which avoids splitting each paragraph into multiple chunks.

Using 6,000 characters per chunk, you can get summaries for each portion separately. The number of tokens, or word pieces, in a chunk depends on the text.

In [ ]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

## Task 2b.5: Summarizing chunks and combining them

In this task, assuming that the number of tokens is consistent in the other documents, you should be good to go. You can use LangChain's `load_summarize_chain` to summarize the text. `load_summarize_chain` provides three ways of summarization: `stuff`, `map_reduce`, and `refine`.

- `stuff`: puts all the chunks into one prompt. Thus, this would hit the maximum limit of tokens.
- `map_reduce`: summarizes each chunk, combines the summaries, and summarizes the combined summary. If the combined summary is too large, it would raise an error.
- `refine`: summarizes the first chunk, and then summarizes the second chunk with the first summary. The same process repeats until all chunks are summarized.

Both map_reduce and refine invoke the LLM multiple times and take time for obtaining the final summary. You can try map_reduce here.

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** Depending on your number of documents, Bedrock request rate quota, and configured retry settings - the chain below may take some time to run.

In [ ]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# print output
print(output['output_text'])

You have now experimented with using prompt chunking and chaining with the LangChain framework to summarize large documents while mitigating issues arising from long input text.

### Try it yourself
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file and continue with **Task 3**.